<a href="https://colab.research.google.com/github/avner8943/bootcamp/blob/master/decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [134]:
import numpy as np

def get_split2(data, feature, split_val):

  right = data[data[:,feature] >= split_val]

  left = data[data[:,feature] < split_val]

  return left, right


def calc_gini(left, right, classes):

  tot_counts = float(len(left) + len(right))



  gini = 0.0

  if left.shape[0] == 0 or right.shape[0] == 0: 
    
    return 1e4


  left_0 = np.count_nonzero(left[:,-1] == class_values[0])/left.shape[0]
  left_1 = np.count_nonzero(left[:,-1] == class_values[1])/left.shape[0]

  right_0 = np.count_nonzero(right[:,-1] == class_values[0])/right.shape[0]
  right_1 = np.count_nonzero(right[:,-1] == class_values[1])/right.shape[0]

  gini = (1 - (left_0**2+left_1**2))*(left.shape[0]/tot_counts) + (1 - (right_0**2+right_1**2))*(right.shape[0]/tot_counts)

  return gini



def get_split(data):

  

  classes = np.unique(data[:,-1])

  best_idx, best_val, best_gini_score, best_groups = 1e3,1e3,1e3,0

  for feature in range(data.shape[1]-1):
    for row in data:
      left, right = get_split2(data, feature, row[feature])
      gini = calc_gini(left, right, classes)
      print('X%d < %.3f Gini=%.3f' % ((feature+1), row[feature], gini),  feature)

      if gini < best_gini_score:
        best_idx, best_val, best_gini_score, best_groups = feature, row[feature], gini, [left, right]
  return {'index':best_idx, 'value':best_val, 'groups':best_groups, 'gini':best_gini_score}
 
dataset = np.array([[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]])
split = get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))




class node():

  def __init__(self, left, right, feature, val, depth, data):

    if left == 0 or right == 0:
      self.is_leaf = 1

    self. left = left
    self.right = right
    self.split_feature = feature
    self.split_val = val
    self.depth = depth
    self.data = data
    return

def split_node(node = 'root', max_depth, min_size, depth):

  if node == 'root':
    root = get_split2(dataset)

  
  if depth > max_depth:

    node.left = node()



X1 < 2.771 Gini=0.444 0
X1 < 1.729 Gini=10000.000 0
X1 < 3.678 Gini=0.286 0
X1 < 3.961 Gini=0.167 0
X1 < 2.999 Gini=0.375 0
X1 < 7.498 Gini=0.286 0
X1 < 9.002 Gini=0.375 0
X1 < 7.445 Gini=0.167 0
X1 < 10.125 Gini=0.444 0
X1 < 6.642 Gini=0.000 0
X2 < 1.785 Gini=0.500 1
X2 < 1.170 Gini=0.444 1
X2 < 2.813 Gini=0.320 1
X2 < 2.620 Gini=0.417 1
X2 < 2.209 Gini=0.476 1
X2 < 3.163 Gini=0.167 1
X2 < 3.339 Gini=0.444 1
X2 < 0.477 Gini=10000.000 1
X2 < 3.235 Gini=0.286 1
X2 < 3.320 Gini=0.375 1
Split: [X1 < 6.642]


In [140]:

# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

# Calculate the Gini index for a split dataset
def gini_index(groups, classes):
	# count all samples at split point
	n_instances = float(sum([len(group) for group in groups]))
	# sum weighted Gini index for each group
	gini = 0.0
	for group in groups:
		size = float(len(group))
		# avoid divide by zero
		if size == 0:
			continue
		score = 0.0
		# score the group based on the score for each class
		for class_val in classes:
			p = [row[-1] for row in group].count(class_val) / size
			score += p * p
		# weight the group score by its relative size
		gini += (1.0 - score) * (size / n_instances)
	return gini

# Select the best split point for a dataset
def get_split(dataset):
	class_values = list(set(row[-1] for row in dataset))
	b_index, b_value, b_score, b_groups = 999, 999, 999, None
	for index in range(len(dataset[0])-1):
		for row in dataset:
			groups = test_split(index, row[index], dataset)
			gini = gini_index(groups, class_values)
			if gini < b_score:
				b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Create a terminal node value
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

# Create child splits for a node or make terminal
def split(node, max_depth, min_size, depth):
	left, right = node['groups']
	del(node['groups'])
	# check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
	# process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left)
		split(node['left'], max_depth, min_size, depth+1)
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right)
		split(node['right'], max_depth, min_size, depth+1)

# Build a decision tree
def build_tree(train, max_depth, min_size):
	root = get_split(train)
	split(root, max_depth, min_size, 1)
	return root

# Print a decision tree
def print_tree(node, depth=0):
	if isinstance(node, dict):
		print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
		print_tree(node['left'], depth+1)
		print_tree(node['right'], depth+1)
	else:
		print('%s[%s]' % ((depth*' ', node)))

dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
tree = build_tree(dataset, 5, 1)
print_tree(tree)


[X1 < 6.642]
 [X1 < 2.771]
  [0]
  [X1 < 2.771]
   [0]
   [0]
 [X1 < 7.498]
  [X1 < 7.445]
   [1]
   [1]
  [X1 < 7.498]
   [1]
   [1]
